*Things we can try:*

- Increase dataset size for train and test using experimental data in aci bench
- use generation args/training args to not let responses repeat(as mentioned by anoshor)
- Improve prompts
- Mess with hyper parameters to tune responses if necessary 

In [118]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
import torch
print(torch.cuda.is_available())  # Should print True if CUDA is available
print(torch.version.cuda)         # Should print the installed CUDA version
print(torch.backends.cudnn.enabled)  # Should be True if cuDNN is enabled

True
11.8
True


In [3]:
!nvidia-smi

Tue Mar 18 17:44:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.61                 Driver Version: 572.61         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000             WDDM  |   00000000:47:00.0 Off |                  Off |
| 30%   35C    P8             21W /  300W |     499MiB /  49140MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
import bitsandbytes as bnb
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
import pandas as pd


C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data_path = "C:\\Users\\BMSCE CSE.DESKTOP-IUB6THA\\Downloads\\kshitij\\clinical_visit_note_summarization_corpus\\data\\aci-bench\\"

train_df = pd.read_csv(data_path + "challenge_data\\train.csv")

In [7]:
train_df.head(1)

,dataset,encounter_id,dialogue,note
0,virtassist,D2N001,"[doctor] hi , martha . how are you ?\r\n[patie...",CHIEF COMPLAINT\r\n\r\nAnnual exam.\r\n\r\nHIS...


In [9]:
print(train_df.iloc[0]['dialogue'])

[doctor] hi , martha . how are you ?
[patient] i'm doing okay . how are you ?
[doctor] i'm doing okay . so , i know the nurse told you about dax . i'd like to tell dax a little bit about you , okay ?
[patient] okay .
[doctor] martha is a 50-year-old female with a past medical history significant for congestive heart failure , depression and hypertension who presents for her annual exam . so , martha , it's been a year since i've seen you . how are you doing ?
[patient] i'm doing well . i've been traveling a lot recently since things have , have gotten a bit lighter . and i got my , my vaccine , so i feel safer about traveling . i've been doing a lot of hiking . uh , went to washington last weekend to hike in northern cascades, like around the mount baker area .
[doctor] nice . that's great . i'm glad to hear that you're staying active , you know . i , i just love this weather . i'm so happy the summer is over . i'm definitely more of a fall person .
[patient] yes , fall foliage is the 

In [11]:
print(train_df.iloc[0]['note'])

CHIEF COMPLAINT

Annual exam.

HISTORY OF PRESENT ILLNESS

Martha Collins is a 50-year-old female with a past medical history significant for congestive heart failure, depression, and hypertension who presents for her annual exam. It has been a year since I last saw the patient.

The patient has been traveling a lot recently since things have gotten a bit better. She reports that she got her COVID-19 vaccine so she feels safer about traveling. She has been doing a lot of hiking.

She reports that she is staying active. She has continued watching her diet and she is doing well with that. The patient states that she is avoiding salty foods that she likes to eat. She has continued utilizing her medications. The patient denies any chest pain, shortness of breath, or swelling in her legs.

Regarding her depression, she reports that she has been going to therapy every week for the past year. This has been really helpful for her. She denies suicidal or homicidal ideation.

The patient reports

In [13]:
train_df.shape

(67, 4)

In [15]:
train_extended_df = pd.read_csv(data_path+"src_experiment_data\\train_virtscribe_asr.csv")

train_extended_df.shape

(12, 4)

In [17]:
train_extended_df1 = pd.read_csv(data_path+"src_experiment_data\\train_aci_asrcorr.csv")

train_extended_df1.shape

(35, 4)

In [21]:
print(train_extended_df.iloc[2]['dialogue'])

[doctor] patrick allen , date of birth 7/7/1977 . new patient visit . past medical history includes gerd , anxiety , depression , here for chronic abdominal pain , he had an abdominal ct on 1/23/2020 . impression is a normal ct of the abdomen . hello , are you mr. allen ? [patient] yes , i am . [doctor] hi , my name is dr. edwards , nice to meet you . [patient] nice to meet you . [doctor] welcome to the gi specialty clinic . [patient] thank you . [doctor] did you have any problems finding this ? [patient] no , i've been here with my sister once before . [doctor] good , so how can i help you today ? the referral i have is for abdominal pain and diarrhea . [patient] right so i've had been having this pain right here in my stomach like right around here . [doctor] so in the area of your mid abdomen just below the belly button ? [patient] correct . i've had the pain on and off for about 2 years . i finally went to the er and a a few months ago and they did a ct scan . [doctor] i saw that .

In [59]:
print(train_extended_df.iloc[0]['note'])

CHIEF COMPLAINT

Hypothyroidism.
Abnormal liver enzymes.
Abnormal weight gain.
Acne vulgaris.

HISTORY OF PRESENT ILLNESS

Christine Hernandez is a 39-year-old female who presents today for evaluation and management of abnormal liver enzymes and hypothyroidism.

When the patient was last seen, her thyroid medicine was changed to Unithroid, but due to the cost of $75, she has not yet started the new medicine and continues taking her previous thyroid medicine. She denies utilizing the coupon that was previously recommended for the Unithroid. Similarly, the patient was prescribed Epiduo for her acne vulgaris, which she did not fill due to the cost of $100. She tried filling both prescriptions at Walmart.

Mrs. Hernandez still presents with abnormal weight gain. She questioned if her oral contraceptive Microgestin, which was discontinued, could have been the cause. Additional symptoms included acne, hirsutism. She is attempting to modify her intake and plans to resume her gym membership in

In [67]:
print(train_extended_df1.iloc[0]['dialogue'])

[doctor] hey george how are you today i understand you're here for some numbness and tingling in your fingers and some pain in your wrist
[patient] right my left wrist and hand has been bothering me probably for a few months now with pain and numbness
[doctor] okay and you said that's been ongoing for several months do you know what caused this type of pain or is it just something that started slowly or
[patient] it just kinda started on it's own it i notice it mostly at night
[doctor] okay
[patient] sometimes it will i'll wake up and my hands asleep and i got ta shake it out
[doctor] shake it out and okay
[patient] and then some
[doctor] what kind of work do you do
[patient] i do yard work
[doctor] yard work
[patient] landscaping landscaping
[doctor] landscaping okay so a lot of raking a lot of digging so a lot of repetitive type movements
[patient] yeah it's pretty heavy labor but it's yeah the same thing day in and day out
[doctor] okay okay just a couple questions for you you did s

In [69]:
print(train_extended_df1.iloc[0]['note'])

CHIEF COMPLAINT

Left wrist and hand pain.

HISTORY OF PRESENT ILLNESS

George Lewis is a pleasant 57-year-old male who presents to the clinic today for evaluation of left wrist and hand pain. He reports an onset of a few months ago but denies any specific injury. However, the patient notes he often engages in repetitive motions while performing his work duties. His symptoms are worse at night, and he wakes with numbness in the bilateral hands. He experiences numbness in all fingers, but states it is the most noticeable in the left thumb and index finger. He affirms intermittent numbness in the left little finger. For relief, he shakes his hands upon waking. The patient also experiences weakness in his left hand. He reports he drops objects and explains “I have a hard time feeling it.”

MEDICAL HISTORY

The patient denies a history of rheumatoid arthritis.

SOCIAL HISTORY

He works in landscaping. He reports consuming 1 to 2 beers on weekends.

REVIEW OF SYSTEMS

Musculoskeletal: Repor

In [77]:
valid_df = pd.read_csv(data_path + "challenge_data\\valid.csv")

print(valid_df.iloc[0]['note'])

CHIEF COMPLAINT

Follow-up of chronic problems.

HISTORY OF PRESENT ILLNESS

Brian White is a 58-year-old male with a past medical history significant for congestive heart failure and hypertension, who presents today for follow-up of his chronic problems.

The patient states he has been feeling out of sorts lately. He is not sure if it is due to the change in the seasons or due to performing lots of projects and some construction on his home. He reports fatigue and lightheadedness. This has been going on for about 5 weeks. While exerting energy, he has experienced some shortness of breath and chest cramps. The patient also notes a slight cough, but he is not sure if it is just the change in seasons.

He feels bloated every once in a while. His diet has been a little bit of a struggle. They had construction on their kitchen begin over Labor Day weekend, and have been eating less healthy food as a result.

Regarding his heart failure, he has been pretty good with his salt intake. He has 

In [85]:
valid_df.shape

(20, 4)

In [83]:
test_df = pd.read_csv(data_path + "challenge_data\\clinicalnlp_taskB_test1.csv")

print(test_df.iloc[0]['dialogue'])

[doctor] hi , andrew . how are you ?
[patient] hey , good to see you .
[doctor] i'm doing well , i'm doing well .
[patient] good .
[doctor] so , i know the nurse told you about dax . i'd like to tell dax a little bit about you .
[patient] sure .
[doctor] uh , so , andrew is a 59-year-old male with a past medical history , significant for depression , type two diabetes , and hypertension who presents today with an upper respiratory infection . so , andrew , what's going on ?
[patient] yeah . we were doing a bit of work out in the yard in the last week or so and i started to feel really tired , was short of breath . um , we- we're not wearing masks as much at the end of the summer and i think i caught my first cold and i think it just got worse .
[doctor] okay . all right . um , now , have you had your covid vaccines ?
[patient] yeah , both .
[doctor] okay . all right . and , um , do you have any history of any seasonal allergies at all ?
[patient] none whatsoever .
[doctor] okay . all r

In [23]:
import os
import pandas as pd

def load_and_print_csv_files(directory):
    # Iterate through all files in the given directory
    for filename in os.listdir(directory):
        # Check if the file is a CSV file and does not contain 'metadata' in the filename
        if filename.endswith('.csv') and 'metadata' not in filename.lower():
            file_path = os.path.join(directory, filename)
            
            try:
                # Load the CSV file using pandas
                df = pd.read_csv(file_path)
                
                # Print the filename, shape of the dataframe, and the columns (features)
                print(f"Filename: {filename}")
                print(f"Shape: {df.shape}")
                print(f"Features: {df.columns.tolist()}")
                print("-" * 50)
            except Exception as e:
                print(f"Failed to load {filename}. Error: {e}")

In [29]:
def print_info(path):
    df = pd.read_csv(data_path + path)
    
    print(df.iloc[0]['dialogue'])
    print(df.iloc[0]['note'])

In [43]:
print_info("challenge_data\\"+"clinicalnlp_taskC_test2.csv")

[doctor] hi , carolyn . how are you ?
[patient] good . how are you ?
[doctor] i'm good . you're ready to get started ?
[patient] yes , i am .
[doctor] so carolyn is a 49-year-old female here for follow-up of her chronic problems . so , how are you doing , carolyn ?
[patient] i'm doing okay . um , my osteoarthritis has been flaring up a little bit lately , um , so i wan na get- talk about that primarily . so , it's mostly in um , my elbows and it's really been bothering me . i've been sitting at the computer , at the desk more , um , in the office so i do n't know if it's being stagnant in that position all day has um , increased flare-ups for that , but i wanted to talk to you about that .
[doctor] okay , sure . and , are you right or left-handed ?
[patient] i'm right-handed .
[doctor] okay , and does one elbow bother you more than the other ?
[patient] my right elbow does bother me more than the other , but primarily , i'm typing all day versus writing , so it- it flares up in both .


In [25]:
load_and_print_csv_files(data_path + "challenge_data")

Filename: clef_taskC_test3.csv
Shape: (40, 4)
Features: ['dataset', 'encounter_id', 'dialogue', 'note']
--------------------------------------------------
Filename: clinicalnlp_taskB_test1.csv
Shape: (40, 4)
Features: ['dataset', 'encounter_id', 'dialogue', 'note']
--------------------------------------------------
Filename: clinicalnlp_taskC_test2.csv
Shape: (40, 4)
Features: ['dataset', 'encounter_id', 'dialogue', 'note']
--------------------------------------------------
Filename: train.csv
Shape: (67, 4)
Features: ['dataset', 'encounter_id', 'dialogue', 'note']
--------------------------------------------------
Filename: valid.csv
Shape: (20, 4)
Features: ['dataset', 'encounter_id', 'dialogue', 'note']
--------------------------------------------------


In [41]:
print_info("src_experiment_data\\"+"test1_aci_asr.csv")

[doctor] hey charles i'm using this cool new recording device to help me with my documentation is that okay with you
[patient] sure
[doctor] awesome how are you doing today
[patient] well i could be better you know i moved out in the city
[doctor] about two years ago bought this big plot of land
[patient] oh
[doctor] i love it lots of deer round awesome
[patient] you be hunter
[doctor] huge hunter i love hunter yeah
[patient] i have a refrigerator full of venison at home
[doctor] do you
[patient] mm-hmm
[doctor] you want to carry share sure yeah we can switch
[patient] nice but so the thing is i have noticed that for some reason my breathing
[doctor] is n't what it used to be
[patient] sure
[doctor] yeah it's you know i i seem to catch my breath a lot more than i used to and i consider myself to be pretty healthy
[patient] you look pretty healthy
[doctor] thanks so do you thank you you you're feeling short of breath
[patient] yes
[doctor] okay is there you know is there any other sympt

In [27]:
load_and_print_csv_files(data_path + "src_experiment_data")

Filename: test1_aci_asr.csv
Shape: (22, 4)
Features: ['dataset', 'id', 'dialogue', 'note']
--------------------------------------------------
Filename: test1_aci_asrcorr.csv
Shape: (22, 4)
Features: ['dataset', 'id', 'dialogue', 'note']
--------------------------------------------------
Filename: test1_virtscribe_asr.csv
Shape: (8, 4)
Features: ['dataset', 'id', 'dialogue', 'note']
--------------------------------------------------
Filename: test1_virtscribe_humantrans.csv
Shape: (8, 4)
Features: ['dataset', 'id', 'dialogue', 'note']
--------------------------------------------------
Filename: test2_aci_asr.csv
Shape: (22, 4)
Features: ['dataset', 'id', 'dialogue', 'note']
--------------------------------------------------
Filename: test2_aci_asrcorr.csv
Shape: (22, 4)
Features: ['dataset', 'id', 'dialogue', 'note']
--------------------------------------------------
Filename: test2_virtscribe_asr.csv
Shape: (8, 4)
Features: ['dataset', 'id', 'dialogue', 'note']
------------------------

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:42:46_Pacific_Standard_Time_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0


In [83]:
import torch
import bitsandbytes as bnb
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset, Dataset


# 1) 4-bit quant config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# 2) Load base model in 4-bit
base_model = AutoModelForCausalLM.from_pretrained(
    "HPAI-BSC/Qwen2.5-Aloe-Beta-7B",
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("HPAI-BSC/Qwen2.5-Aloe-Beta-7B")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# 3) Apply LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], 
)
model = get_peft_model(base_model, lora_config)

# 4) Enable gradient checkpointing
model.enable_input_require_grads()
model.gradient_checkpointing_enable()
base_model.config.use_cache = False

# 5) Prepare data with smaller sequence length
# notechat = load_dataset("akemiH/NoteChat")
# trainB = notechat["train"].select(range(3000))
# evalB = notechat["train"].select(range(3000, 3500))

train_df = pd.read_csv(data_path+"\\challenge_data"+"\\train.csv")
train_df2 = pd.read_csv(data_path+"\\challenge_data"+"\\valid.csv")
train_df3 = pd.read_csv(data_path+"\\challenge_data"+"\\clef_taskC_test3.csv")
train_df4 = pd.read_csv(data_path+"\\challenge_data"+"\\clinicalnlp_taskB_test1.csv")

trainB = pd.concat([train_df, train_df2, train_df3, train_df4], axis=0, ignore_index=True)
evalB = pd.read_csv(data_path + "\\challenge_data" + "\\clinicalnlp_taskC_test2.csv")

print("Shape of the combined dataframe:", trainB.shape)
print("Features in the combined dataframe:", trainB.columns.tolist())

MAX_LENGTH = 256  # smaller sequence length

class ConversationSummaryDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, tokenizer, conversation, summary, max_length=MAX_LENGTH):
        self.data = hf_dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.conversation = conversation
        self.summary = summary

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if isinstance(self.data, Dataset):
            conversation = str(self.data[idx][self.conversation])
            summary = str(self.data[idx][self.summary])
        elif isinstance(self.data, pd.DataFrame):
            conversation = str(self.data.iloc[idx][self.conversation])
            summary = str(self.data.iloc[idx][self.summary])
        else:
            raise TypeError("Data must either be a pandas dataframe or a huggingface dataset")
            
        text = f"CONVERSATION:\n{conversation}\n\nSUMMARY: {summary}"

        encodings = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt",
        )
        input_ids = encodings["input_ids"].squeeze()
        attention_mask = encodings["attention_mask"].squeeze()

        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

train_dataset = ConversationSummaryDataset(trainB, tokenizer, "dialogue", "note")
eval_dataset = ConversationSummaryDataset(evalB, tokenizer, "dialogue", "note")

Loading checkpoint shards: 100%|███████████████████████████████████████████████| 4/4 [00:12<00:00,  3.03s/it]


Shape of the combined dataframe: (167, 4)
Features in the combined dataframe: ['dataset', 'encounter_id', 'dialogue', 'note']


In [85]:
# 6) Training arguments
training_args = TrainingArguments(
    output_dir="./aloe_qwen_aci-bench",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,      # accumulate to compensate for small batch
    evaluation_strategy="epoch",       # you can skip or reduce evaluation
    fp16=True,
    logging_steps=50,
    save_strategy="epoch",
    save_total_limit=2,
    dataloader_pin_memory=True,
    dataloader_drop_last=True,
    gradient_checkpointing=True,       # from the Trainer side as well
)

# 7) Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# 8) Train
trainer.train()

C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\BMSCE CSE.DESKTOP-IUB6THA\AppData\Local\Temp\ipykernel_20020\2075795087.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,No log,2.074234


TrainOutput(global_step=40, training_loss=2.143132972717285, metrics={'train_runtime': 156.6595, 'train_samples_per_second': 2.132, 'train_steps_per_second': 0.255, 'total_flos': 3466962839273472.0, 'train_loss': 2.143132972717285, 'epoch': 1.910179640718563})

In [87]:
model.save_pretrained("./aloe_qwen_aci-bench")
tokenizer.save_pretrained("./aloe_qwen_aci-bench")

('./aloe_qwen_aci-bench\\tokenizer_config.json',
 './aloe_qwen_aci-bench\\special_tokens_map.json',
 './aloe_qwen_aci-bench\\vocab.json',
 './aloe_qwen_aci-bench\\merges.txt',
 './aloe_qwen_aci-bench\\added_tokens.json',
 './aloe_qwen_aci-bench\\tokenizer.json')

In [95]:
import torch
import bitsandbytes as bnb
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from peft import PeftModel

model_dir = "./aloe_qwen_aci-bench"

# -- Path to your LoRA weights + tokenizer --
if isinstance(train_dataset, Dataset):
    model_dir = "./aloe_qwen"
    print("check")
elif isinstance(train_dataset, pd.DataFrame):
    model_dir = "./aloe_qwen_aci-bench"
    
print(model_dir)

# -- 4-bit quantization config (same as training) --
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# -- 1) Load the *base* Qwen2.5 model in 4-bit --
base_model = AutoModelForCausalLM.from_pretrained(
    "HPAI-BSC/Qwen2.5-Aloe-Beta-7B",
    quantization_config=bnb_config,
    device_map="auto",
)

# -- 2) Load your fine-tuned LoRA adapters into the base model --
# The directory should contain adapter_model.bin, adapter_config.json, etc.
lora_model = PeftModel.from_pretrained(
    base_model, 
    model_dir,
    device_map="auto",
)

# -- 3) Load the tokenizer you saved to ./aloe_qwen --
tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


./aloe_qwen_aci-bench


Loading checkpoint shards: 100%|███████████████████████████████████████████████| 4/4 [00:09<00:00,  2.47s/it]


In [5]:
### FOR NoteChat testing model

from transformers import pipeline

# 1) Create a conversation + summary prompt
conversation_text = """
Doctor: Hi, Mr. X, I'm Dr. Y. How are you feeling today?
Patient: Not too good, doctor. I've been feeling really sick lately.
Doctor: I understand. Can you tell me what symptoms you're experiencing?
Patient: Yes, I've been having a fever, a dry cough, and dyspnea.
Doctor: I see. You were hospitalized due to moderate ARDS from COVID-19, is that correct?
Patient: Yes, that's correct.
Doctor: During your physical therapy, we encountered some difficulties. Can you tell me more about that?
Patient: Yes, I had trouble with position changes and deep breathing. Every time I tried to change my position or take a deep breath, I would start coughing and it would make me really short of breath.
Doctor: I understand. To avoid rapid deterioration and respiratory failure, we instructed you to change positions very slowly and step-by-step, right?
Patient: Yes, that's right.
Doctor: And I see that this approach increased your oxygen saturation, for example, on day 5 with 6 L/min of oxygen from 93% to 97%.
Patient: Yes, that's correct.
Doctor: Good. We also had to adapt your breathing exercises to avoid prolonged coughing and oxygen desaturation. Can you tell me more about that?
Patient: Yes, I was instructed to stop every deep breath before coughing and to hold my breath for better air distribution.
Doctor: I see that you performed the breathing exercises well and managed to increase your oxygen saturation.
Patient: Yes, I did my best.
Doctor: You also had difficulty maintaining sufficient oxygen saturation during physical activity, is that correct?
Patient: Yes, I did. But with close monitoring and frequent breaks, I was able to perform low-level strength and walking exercises without any significant deoxygenation.
Doctor: I see that your exercise progression was low on days 1 to 5, but then increased daily until your hospital discharge to a rehabilitation clinic on day 10.
Patient: Yes, that's correct.
Doctor: Great. I'd like to keep monitoring your progress and see how you're doing. Can you keep me updated on any changes in your symptoms?
Patient: Yes, of course, doctor.
Doctor: Alright, let's keep in touch. If you have any questions or concerns, don't hesitate to reach out to me.
Patient: Thank you, doctor.
"""
prompt = f"""
CONVERSATION:
{conversation_text}

SUMMARY:
- **Symptoms**:
  (Include symptoms mentioned in the conversation, if any.)

- **Physical Activities**:
  (Include details about physical activities or excercise habits mentioned in the conversation, if any.)

- **Medical History**:
  (Include any relavent medical history mentioned in the conversation, if any.)

- **Surgical History**
  (Include any details about past surgeries mentioned in the conversation, if any.)

- **Treatment Plan History**
  (Include any details about past treatment plans or suggestions made by healthcare professionals, including medications, surgeries, therapies, or lifestyle changes, if any.)

Provide the summary in plain text with the above attributes, and include information only if it is mentioned in the conversation. If any attribute is not discussed, omit it. Only include the summary in the generated response.Format the summary properly.\n\n
"""

# Create a text-generation pipeline
# (You can name it 'summarizer' if you wish, but it's using text-generation.)
text_gen_pipeline = pipeline(
    "text-generation",
    model=lora_model,
    tokenizer=tokenizer,
    device_map="auto",   # place on GPU if available
)
# Now generate text using the pipeline
response = text_gen_pipeline(
    prompt,
    max_new_tokens=400,
    do_sample=False,  # Deterministic for demonstration
)

# The output is a list of dict(s); each has "generated_text"
generated_text = response[0]["generated_text"]
# print("FULL OUTPUT:\n", generated_text)

# If the model replicates your training format, you'll see something like:
# "CONVERSATION: ... SUMMARY: <the actual summary text>"
# You can strip off everything before "SUMMARY:" if you only want the summary.
if "Summary:" in generated_text:
    final_summary = generated_text.split("Summary:", 1)[-1].strip()
else:
    final_summary = generated_text

print("\nEXTRACTED SUMMARY:\n", final_summary)


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausa


EXTRACTED SUMMARY:
 - **Symptoms**: Fever, dry cough, dyspnea
- **Physical Activities**: Position changes, deep breathing, breathing exercises, strength and walking exercises
- **Medical History**: No relevant information provided
- **Surgical History**: No relevant information provided
- **Treatment Plan History**: Physical therapy, gradual position changes, breathing exercises adapted to avoid coughing and oxygen desaturation, low-level strength and walking exercises with frequent breaks to prevent deoxygenation. Exercise progression increased daily until hospital discharge to a rehabilitation clinic. Doctor will continue to monitor patient's progress and update on any changes in symptoms. 1. Patient presented with symptoms of fever, dry cough, and dyspnea.
2. He was diagnosed with moderate ARDS from COVID-19 and hospitalized.
3. During physical therapy, he experienced difficulties with position changes and deep breathing, leading to coughing and shortness of breath.
4. To avoid rap

In [9]:
conversation_text = """
Doctor: Hello, I am Dr. Smith. Can you tell me what brings you to the hospital today?
Patient: Yes, I have been feeling very weak and sick for the past two weeks. I have a persistent fever and dry cough.
Doctor: I see. And how is your breathing?
Patient: It's been shallow and rapid, especially when I am at rest. And I get severely breathless even with minor physical activities.
Doctor: Okay. I understand. You were given physical therapy, right?
Patient: Yes, they focused on educating me about dyspnea-relieving positions and the importance of regular mobilization and deep-breathing exercises.
Doctor: That's good. And how did it go?
Patient: It became evident that my anxiety from fear of dying and worries about my future was making my dyspnea worse. I was so dyspneic, anxious, and weak that I could barely walk to the toilet.
Doctor: I see. But your physical therapist helped you with that, right?
Patient: Yes, they listened to me, explained why I was experiencing breathlessness, and tested suitable positions to relieve my dyspnea. I felt better after the education and relaxing breathing exercises.
Doctor: That's great to hear. Can you tell me more about the improvement?
Patient: On day 2, my respiratory rate reduced from 30 breaths/min to 22 breaths/min and my oxygen saturation increased from 92% to 96% on 4 L/min oxygen after some deep-breathing exercises.
Doctor: That's impressive. And how did it go after that?
Patient: My dyspnea and anxiety started to alleviate and I regained my self-confidence. The therapy was shifted to walking and strength training, and I was able to walk 350 m without a walking aid or supplemental oxygen before my discharge home.
Doctor: That's fantastic. You have made a great progress. I am glad to hear that.
"""
prompt = f"""
CONVERSATION:
{conversation_text}

SUMMARY:
- **Symptoms**:
  (Include symptoms mentioned in the conversation, if any.)

- **Physical Activities**:
  (Include details about physical activities or excercise habits mentioned in the conversation, if any.)

- **Medical History**:
  (Include any relavent medical history mentioned in the conversation, if any.)

- **Surgical History**
  (Include any details about past surgeries mentioned in the conversation, if any.)

- **Treatment Plan History**
  (Include any details about past treatment plans or suggestions made by healthcare professionals, including medications, surgeries, therapies, or lifestyle changes, if any.)

Provide the summary in plain text with the above attributes, and include information only if it is mentioned in the conversation. If any attribute is not discussed, omit it. Only include the summary in the generated response.Format the summary properly.\n\n
"""

response = text_gen_pipeline(
    prompt,
    max_new_tokens=500,
    do_sample=False,  # Deterministic for demonstration
)

# The output is a list of dict(s); each has "generated_text"
generated_text = response[0]["generated_text"]
# print("FULL OUTPUT:\n", generated_text)

# If the model replicates your training format, you'll see something like:
# "CONVERSATION: ... SUMMARY: <the actual summary text>"
# You can strip off everything before "SUMMARY:" if you only want the summary.
if "SUMMARY:" in generated_text:
    final_summary = generated_text.split("SUMMARY:", 1)[-1].strip()
else:
    final_summary = generated_text

print("\nEXTRACTED SUMMARY:\n", final_summary)


EXTRACTED SUMMARY:
 - **Symptoms**:
  (Include symptoms mentioned in the conversation, if any.)

- **Physical Activities**:
  (Include details about physical activities or excercise habits mentioned in the conversation, if any.)

- **Medical History**:
  (Include any relavent medical history mentioned in the conversation, if any.)

- **Surgical History**
  (Include any details about past surgeries mentioned in the conversation, if any.)

- **Treatment Plan History**
  (Include any details about past treatment plans or suggestions made by healthcare professionals, including medications, surgeries, therapies, or lifestyle changes, if any.)

Provide the summary in plain text with the above attributes, and include information only if it is mentioned in the conversation. If any attribute is not discussed, omit it. Only include the summary in the generated response.Format the summary properly.


Summary: The patient presented with a persistent fever, dry cough, and severe breathlessness. Sh

In [123]:
#### FOR ACI BENCH TESTING MODEL


from transformers import pipeline

# 1) Create a conversation + summary prompt
conversation_text = """
Doctor: Hi, Mr. X, I'm Dr. Y. How are you feeling today?
Patient: Not too good, doctor. I've been feeling really sick lately.
Doctor: I understand. Can you tell me what symptoms you're experiencing?
Patient: Yes, I've been having a fever, a dry cough, and dyspnea.
Doctor: I see. You were hospitalized due to moderate ARDS from COVID-19, is that correct?
Patient: Yes, that's correct.
Doctor: During your physical therapy, we encountered some difficulties. Can you tell me more about that?
Patient: Yes, I had trouble with position changes and deep breathing. Every time I tried to change my position or take a deep breath, I would start coughing and it would make me really short of breath.
Doctor: I understand. To avoid rapid deterioration and respiratory failure, we instructed you to change positions very slowly and step-by-step, right?
Patient: Yes, that's right.
Doctor: And I see that this approach increased your oxygen saturation, for example, on day 5 with 6 L/min of oxygen from 93% to 97%.
Patient: Yes, that's correct.
Doctor: Good. We also had to adapt your breathing exercises to avoid prolonged coughing and oxygen desaturation. Can you tell me more about that?
Patient: Yes, I was instructed to stop every deep breath before coughing and to hold my breath for better air distribution.
Doctor: I see that you performed the breathing exercises well and managed to increase your oxygen saturation.
Patient: Yes, I did my best.
Doctor: You also had difficulty maintaining sufficient oxygen saturation during physical activity, is that correct?
Patient: Yes, I did. But with close monitoring and frequent breaks, I was able to perform low-level strength and walking exercises without any significant deoxygenation.
Doctor: I see that your exercise progression was low on days 1 to 5, but then increased daily until your hospital discharge to a rehabilitation clinic on day 10.
Patient: Yes, that's correct.
Doctor: Great. I'd like to keep monitoring your progress and see how you're doing. Can you keep me updated on any changes in your symptoms?
Patient: Yes, of course, doctor.
Doctor: Alright, let's keep in touch. If you have any questions or concerns, don't hesitate to reach out to me.
Patient: Thank you, doctor.
"""
prompt = f"""
CONVERSATION:
{conversation_text}

SUMMARY:
Please generate a medical summary based on the following clinical notes. The summary should include the following sections: Chief Complaint, History of Present Illness, Vitals, Physical Exam, Results, Assessment, Plan, and Instructions.
Please format the response as plain text, without using markdown or special formatting, and with clear headings for each section, like this:


CHIEF COMPLAINT
A concise statement of the patient's primary concern or reason for visiting the clinic.

HISTORY OF PRESENT ILLNESS
A detailed narrative about the patient's symptoms, their onset, duration, and any relevant medical history or previous treatments.

VITALS
Include any relevant vital signs (e.g., oxygen saturation, blood pressure) if available.

PHYSICAL EXAM 
Summarize the findings from the physical examination, including any notable abnormalities.

RESULTS 
Summarize the results of any diagnostic tests performed (e.g., lab work, imaging studies).

ASSESSMENT
The doctor's assessment of the patient's condition or diagnosis.

PLAN
The treatment plan, including prescribed medications, lifestyle recommendations, and follow-up instructions.

INSTRUCTIONS
Specific instructions for the patient regarding their treatment plan and follow-up care.

Important Note: If any section lacks relevant information or if specific details are not provided (e.g., vitals are not mentioned, no abnormal findings in the physical exam), omit that section from the generated summary. Only include sections for which there is sufficient information available.
"""


# Create a text-generation pipeline
# (You can name it 'summarizer' if you wish, but it's using text-generation.)
text_gen_pipeline = pipeline(
    "text-generation",
    model=lora_model,
    tokenizer=tokenizer,
    device_map="auto",   # place on GPU if available
)
# Now generate text using the pipeline
response = text_gen_pipeline(
    prompt,
    max_new_tokens=600,
    do_sample=False,  # Deterministic for demonstration
)

# The output is a list of dict(s); each has "generated_text"
generated_text = response[0]["generated_text"]
# print("FULL OUTPUT:\n", generated_text)

# If the model replicates your training format, you'll see something like:
# "CONVERSATION: ... SUMMARY: <the actual summary text>"
# You can strip off everything before "SUMMARY:" if you only want the summary.
if "Summary:" in generated_text:
    final_summary = generated_text.split("Summary:", 1)[-1].strip()
else:
    final_summary = generated_text

print("\nEXTRACTED SUMMARY:\n", final_summary)


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausa


EXTRACTED SUMMARY:
 
CONVERSATION:

Doctor: Hi, Mr. X, I'm Dr. Y. How are you feeling today?
Patient: Not too good, doctor. I've been feeling really sick lately.
Doctor: I understand. Can you tell me what symptoms you're experiencing?
Patient: Yes, I've been having a fever, a dry cough, and dyspnea.
Doctor: I see. You were hospitalized due to moderate ARDS from COVID-19, is that correct?
Patient: Yes, that's correct.
Doctor: During your physical therapy, we encountered some difficulties. Can you tell me more about that?
Patient: Yes, I had trouble with position changes and deep breathing. Every time I tried to change my position or take a deep breath, I would start coughing and it would make me really short of breath.
Doctor: I understand. To avoid rapid deterioration and respiratory failure, we instructed you to change positions very slowly and step-by-step, right?
Patient: Yes, that's right.
Doctor: And I see that this approach increased your oxygen saturation, for example, on day 5

In [7]:
import evaluate
from tqdm import tqdm

# rouge = evaluate.load("rouge")

In [2]:
import torch
import bitsandbytes as bnb
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from peft import PeftModel

# -- Path to your LoRA weights + tokenizer --
model_dir = "./aloe_qwen"  

# -- 4-bit quantization config (same as training) --
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# -- 1) Load the *base* Qwen2.5 model in 4-bit --
base_model = AutoModelForCausalLM.from_pretrained(
    "HPAI-BSC/Qwen2.5-Aloe-Beta-7B",
    quantization_config=bnb_config,
    device_map="auto",
)

# -- 2) Load your fine-tuned LoRA adapters into the base model --
# The directory should contain adapter_model.bin, adapter_config.json, etc.
lora_model = PeftModel.from_pretrained(
    base_model, 
    model_dir,
    device_map="auto",
)

# -- 3) Load the tokenizer you saved to ./aloe_qwen --
tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.71s/it]


In [ ]:
from transformers import pipeline
summarizer = pipeline(
    "text-generation",
    model=lora_model,
    tokenizer=tokenizer,
    device_map="auto",  # Automatically place on GPU if available
)

notechat = load_dataset("akemiH/NoteChat")
eval_df = notechat["train"].select(range(5000, 7001)).to_pandas()

# 2) Clean up missing data if present
if eval_df.isnull().values.any():
    print("Found missing values in the evaluation set. Dropping them.")
    eval_df = eval_df.dropna()

# 3) Prepare batching parameters
batch_size = 4
num_samples = len(eval_df)
num_batches = (num_samples // batch_size) + int(num_samples % batch_size != 0)

# Lists to store predictions and references
predictions = []
references = []

# 4) Batching loop to generate summaries
for i in tqdm(range(num_batches), desc="Generating Summaries"):
    start = i * batch_size
    end = min(start + batch_size, num_samples)
    
    # Extract conversation and reference summary columns
    batch_conversations = eval_df["conversation"][start:end].tolist()
    batch_refs = eval_df["data"][start:end].tolist()  # "data" column for references
    
    # Prepare prompts replicating training format
    prompts = [
        f"CONVERSATION:\n{conv}\n\nSUMMARY:"
        for conv in batch_conversations
    ]
    
    # Generate summaries
    results = summarizer(
        prompts,
        max_new_tokens=100,
        do_sample=False,
        truncation=True,
        num_return_sequences=1  # Default is 1
    )
    
    # Parse results and extract summaries
    for item in results:
        # 'item' is a list with 1 dict => {"generated_text": "..."}
        output_dict = item[0]
        full_text = output_dict["generated_text"]
        
        # Extract only the part after "SUMMARY:"
        if "SUMMARY:" in full_text:
            extracted_summary = full_text.split("SUMMARY:", 1)[-1].strip()
        else:
            extracted_summary = full_text  # Fallback if marker not found
        
        predictions.append(extracted_summary)
    
    # Append the reference texts
    references.extend(batch_refs)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'Jam

In [ ]:
# Compute ROUGE scores
results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

print("ROUGE scores:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")

###### predictions[0]

In [12]:
predictions[1021]

'The patient presented with a 12-month history of a purple nodular rash on her nasolabial folds. She reported no itching, burning, pain, or bleeding from the area, but did report drainage of clear fluid when applying pressure to the area. She has a history of cirrhosis, iron deficiency anemia, type II diabetes mellitus, hypertension, asthma, and endometrioid carcinoma of the ovary. A 3-mm punch biopsy of the right nasolab'

In [15]:
src_list = eval_df["conversation"].tolist()
ref_list = eval_df["data"].tolist()

In [16]:
import sys
sys.path.append("/home/jovyan/UniEval") 
from utils import convert_to_json
from metric.evaluator import get_evaluator
data = convert_to_json(
    src_list=src_list,
    ref_list=ref_list,
    output_list=output_list
)


In [13]:
output_list = []
for pred in predictions:
    # Ensure that "Summary:" exists in the string to avoid errors
    if len(pred) > 0:
        output_list.append(pred)
    else:
        # Handle cases where "Summary:" is missing (optional)
        output_list.append("")  # Or handle differently based on your needs

In [17]:
filtered_data = [
    entry for entry in data
    if entry["system_output"].strip()  # Ensure non-empty system_output
]

In [ ]:
filtered_data

In [18]:
import sys
sys.path.append("/home/jovyan/UniEval") 
from utils import convert_to_json
from metric.evaluator import get_evaluator
task = "summarization"
evaluator = get_evaluator(task)


In [20]:
eval_scores = evaluator.evaluate(filtered_data, print_result=True)

Evaluating coherence of 2001 samples !!!


100%|██████████| 251/251 [05:44<00:00,  1.37s/it]


Evaluating consistency of 2001 samples !!!


100%|██████████| 1171/1171 [21:27<00:00,  1.10s/it]


Evaluating fluency of 2001 samples !!!


100%|██████████| 1171/1171 [01:30<00:00, 12.97it/s]


Evaluating relevance of 2001 samples !!!


100%|██████████| 251/251 [03:41<00:00,  1.14it/s]


Evaluation scores are shown below:
+-------------+----------+
|  Dimensions |  Score   |
+-------------+----------+
|  coherence  | 0.966174 |
| consistency | 0.884436 |
|   fluency   | 0.886275 |
|  relevance  | 0.820143 |
|   overall   | 0.889257 |
+-------------+----------+


In [19]:
import json

# Suppose `filtered_data` is the final list of dicts you want to store:
# filtered_data = [
#   {"source": "...", "reference": "...", "system_output": "..."},
#   ...
# ]

with open("evaluation_data1.json", "w", encoding="utf-8") as f:
    # Write list of dictionaries to JSON file
    json.dump(filtered_data, f, indent=2, ensure_ascii=False)

print("Data saved to evaluation_data1.json")


Data saved to evaluation_data1.json


In [ ]:
import json

with open("evaluation_data.json", "r", encoding="utf-8") as f:
    filtered_data = json.load(f)

print("Data loaded. Number of samples:", len(filtered_data))
